# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Symbols and Expressions

One of the most basic objects in this approach are unevaluated *symbols*:

In [ ]:
:a  # "the symbol a"

In [ ]:
typeof(:a)

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [ ]:
eval(:a)

`a` must be defined for this to work:

In [ ]:
a = 3

In [ ]:
eval(:a)

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Operators and function names are also symbols:

In [ ]:
:+, :sin

In [ ]:
typeof(:+)

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [ ]:
ex = :(a + b)  # the expression 'a + b'

In [ ]:
typeof(ex)

In [ ]:
b = 7
eval(ex)

An expression is just a Julia object, so we can introspect (find out information about it):

In [ ]:
dump(ex)

In [ ]:
dump(:(x = 3))

In [ ]:
# ex.<TAB>

In [ ]:
ex.head

In [ ]:
ex.args

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [ ]:
parse("a + b")

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [ ]:
ex = :( sin(3a + 2b^2) )

In [ ]:
ex.args

In [ ]:
typeof(ex.args[2])

In [ ]:
ex.args[2].args

In [ ]:
ex.args[1] = :cos

In [ ]:
ex

In [ ]:
blk = quote
    println("Hello")
end

In [ ]:
eval(blk)

In [ ]:
push!(blk.args, :(println("AFTER")))

In [ ]:
blk

In [ ]:
eval(blk)

In [ ]:
unshift!(blk.args, :(println("BEFORE")))

In [ ]:
eval(blk)

Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [ ]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

In [ ]:
eval(ex2)
y

In [ ]:
eval(ex2)
z

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [ ]:
Meta.show_sexpr(ex2)

Another way of seeing the structure is with `dump`:

In [ ]:
dump(ex2)

## Interpolation

Values can be *interpolated* into expressions with `$`.

In [ ]:
x = 2

In [ ]:
:(x + $x)

In [ ]:
x = :y

In [ ]:
:(x + $x)

### Why not strings?

*Abstract* syntax vs. *concrete* syntax.

In [ ]:
a = "x + y"

In [ ]:
b = "z"

In [ ]:
expr = "$a * z"

Even worse, what if `z == ")"`?

With `Expr` objects, every expression is well-formed and each slot has its abstract meaning.

### Splat interpolation

In [ ]:
vars = [:a, :b, :c]

In [ ]:
:(1 - f(0, $(vars...)))

### `@eval` for interpolating on the fly

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately before executing.

In [ ]:
fnames = [ Symbol("func$i") for i=1:2]

In [ ]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = $i
end

In [ ]:
func1(0)

In [ ]:
func2(0)

## Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
    - e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [ ]:
@time sin(10)

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [ ]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

In [ ]:
x = 0
@twice println(x += 1)

In [ ]:
ex = :(@twice println(x += 1))

In [ ]:
eval(ex)

In [ ]:
typeof(ex)

We can see what effect the macro actually has using `macroexpand`:

In [ ]:
macroexpand(:(@twice println(x += 1)))

In [ ]:
macroexpand(:(@time sin(10)))

In [ ]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

In [ ]:
@mytime (sleep(1); "done")

A line break terminates macro arguments, so blocks need to be passed with `begin...end`:

In [ ]:
@time begin
    # code
end

----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

### Macro Hygiene

In [ ]:
macro set_x(val)
    :(x = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

In [ ]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [ ]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [ ]:
x = 3
@horner(x, 2, 3, 4, 5)

To see what the macro does to this call, we again use `macroexpand`:

In [ ]:
macroexpand(:(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

In [ ]:
macroexpand(:(@Base.Math.horner(x, 2, 3, 4, 5)))

In [ ]:
?muladd

In [ ]:
?fma

In [ ]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

In [ ]:
@code_llvm f(0.1)

In [ ]:
@code_native f(0.1)

## `@generated` Functions

Another form of metaprogramming available in Julia.

Generate code based on *types* instead of the input expressions.

In [ ]:
@generated function nloops(a::Array{T,N}) where {T,N}
    # inside here,
    # - `a` refers to the type of the argument
    # - `T` and `N` have their usual values
    # - return an expression to compute the answer for this type
end